In [ ]:
import hyperspy.api as hs
import matplotlib.pyplot as plt

In [ ]:
%matplotlib qt

In [ ]:
path = 'data/Maehlum_2022-09-06_EDS-SEM-APREO'
file30 = 'GaAs_30kV.emsa'
elements = ['Ga', 'As']
zero_peak_end_index = 33

In [ ]:
def do_calibration(lines, calibrate_lines=False):
    s = hs.load(path + '/' + file30, signal_type='EDS_SEM')
    s.add_elements(elements)
    s = s.isig[zero_peak_end_index:]

    # make model
    m = s.create_model()
    m.fit()
    m.fit_background() # makes the bg fit better

    print_str = f'{s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa:.2f} eV -->'

    # calibrate the lines, usually changes the resolution +/- 1 eV
    if calibrate_lines:
        m.calibrate_xray_lines(calibrate='energy', xray_lines='all', kind='multi', iterpath='serpentine')
        m.calibrate_xray_lines(calibrate='width', xray_lines='all', kind='multi', iterpath='serpentine')

    # now calibrate on the requested lines, or 'all_alpha'
    m.calibrate_energy_axis(calibrate='resolution', xray_lines=lines)

    print(f'{print_str} {s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa:.2f} eV')

    return s, m

In [ ]:
s, m = do_calibration(['Ga_La', 'Ga_Ka', 'As_La'], calibrate_lines=True)

In [ ]:
m.calibrate_energy_axis(calibrate='scale', xray_lines=['Ga_La', 'Ga_Ka', 'As_La'])
m.calibrate_energy_axis(calibrate='offset', xray_lines=['Ga_La', 'Ga_Ka', 'As_La'])


In [ ]:
hs.material.elements.Ga.Atomic_properties.Xray_lines.Ka['energy (keV)']

In [ ]:
plt.vlines(m['Ga_Ka'].centre.value, 0, 100000, color='r')
plt.vlines(m['As_Ka'].centre.value, 0, 100000, color='r')
plt.vlines(hs.material.elements.Ga.Atomic_properties.Xray_lines.Ka['energy (keV)'], 0, 100000, color='g')
plt.vlines(hs.material.elements.As.Atomic_properties.Xray_lines.Ka['energy (keV)'], 0, 100000, color='g')


In [ ]:
s.data.shape

In [ ]:
plt.show()

In [ ]:
m.plot(plot_components=True)

In [ ]:
s = hs.load(path + '/' + file30, signal_type='EDS_SEM')
s.add_elements(elements)
s = s.isig[zero_peak_end_index:]
# make model
m = s.create_model()
m.fit()
m.fit_background() # makes the bg fit better

In [ ]:
m.plot(plot_components=True)

In [ ]:

m.calibrate_xray_lines(calibrate='energy', xray_lines='all', kind='multi', iterpath='serpentine')
m.calibrate_xray_lines(calibrate='width', xray_lines='all', kind='multi', iterpath='serpentine')

# now calibrate on the requested lines, or 'all_alpha'
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_Ka', 'As_Ka', 'Ga_Kb', 'As_Kb'])
m.calibrate_energy_axis(calibrate='scale', xray_lines=['Ga_Ka', 'As_Ka', 'Ga_Kb', 'As_Kb'])
m.calibrate_energy_axis(calibrate='offset', xray_lines=['Ga_Ka', 'As_Ka', 'Ga_Kb', 'As_Kb'])


In [ ]:
s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_Kb', 'As_Kb', 'As_Ka', 'Ga_Ka'])
s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa


In [ ]:
path = 'data/Skomedal_2022-03-23_EDS_SU9000'
file = 'Spectrum 03.emsa'  # 30 kV
elements = ['Al', 'Au', 'C', 'Cu', 'F', 'Fe', 'Mg', 'Mo', 'Ni', 'O', 'Si', 'Sn']  # from Mari, pluss Mg and Sn

s = hs.load(path + '/' + file, signal_type='EDS_SEM')
s.add_elements(elements)
s = s.isig[zero_peak_end_index:]
# make model
m = s.create_model()
m.fit()
m.fit_background() # makes the bg fit better

In [ ]:
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Si_Ka'])


In [ ]:
s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
m.xray_lines

In [ ]:
def test_multiple_lines(lines_to_test):
    results = []
    for lines in lines_to_test:
        print(f'{lines}')
        s, m = do_calibration(lines)
        results.append([lines, s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa])
    return results

In [ ]:
lines_to_test = ['all_alpha', ['As_Ka'], ['As_La'], ['Ga_Ka'], ['Ga_La']]
lines_to_test2 = [['As_Ka', 'As_La', 'Ga_Ka', 'Ga_La'], ['Ga_La', 'As_Ka', 'As_La', 'Ga_Ka'], ['Ga_Ka', 'Ga_La', 'As_Ka', 'As_La']]
results = test_multiple_lines(lines_to_test)
results2 = test_multiple_lines(lines_to_test2)

In [ ]:
test_3 = [['Ga_La'], ['Ga_La', 'As_Ka'], ['Ga_La', 'As_Ka', 'As_La'], ['Ga_La', 'As_Ka', 'As_La', 'Ga_Ka'], ['Ga_La', 'As_Ka', 'As_La', 'Ga_Ka', 'Ga_Ll', 'Ga_Kb']]
results3 = test_multiple_lines(test_3)

In [ ]:
# testing As_Ka
test_4 = [['As_Ka'], ['As_Ka', 'Ga_Kb'], ['As_Ka', 'Ga_La', 'As_La', 'Ga_Ka'], ['As_Ka', 'Ga_La', 'As_La', 'Ga_Ka', 'Ga_Kb']]
results4 = test_multiple_lines(test_4)